# Lesson 4: Text-to-SQL RAG Agent

In this lesson, we'll learn how to create an AI agent that can convert natural language queries into SQL and execute them against a database. This approach enables users to interact with databases using plain English instead of writing SQL queries.

We'll build a complete text-to-SQL solution that can handle complex queries across multiple tables in an e-commerce database.

## Lesson Objectives

By the end of this lesson, you will be able to:

1. Set up and populate a SQLite database from CSV data
2. Create basic text-to-SQL conversions using LLMs
3. Build database schema understanding for the AI agent
4. Implement SQL execution with safety measures
5. Use function calling to create dynamic text-to-SQL agents
6. Handle complex multi-table queries and relationships
7. Create a conversational database interface with error handling

## 1. Environment Setup

First, let's set up our environment and generate the sample data.

In [1]:
# Install required packages if needed
# !pip install faker pandas sqlite3 openai python-dotenv

In [2]:
# Import required libraries
import sqlite3
import pandas as pd
import os
import json
from dotenv import load_dotenv, find_dotenv
from openai import AzureOpenAI
from IPython.display import display, HTML, Markdown
import subprocess
import sys

# Load environment variables
load_dotenv(find_dotenv())

True

In [3]:
# Get API keys and configuration
API_KEY = os.environ.get("AZURE_OPENAI_KEY") 
API_ENDPOINT = os.environ.get("AZURE_OPENAI_ENDPOINT")
AZURE_DEPLOYMENT = os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME")
API_VERSION = os.environ.get("AZURE_OPENAI_VERSION")
MODEL = os.environ.get("AZURE_OPENAI_MODEL")

# Check if the necessary API keys are available
if not API_KEY:
    print("⚠️ Azure OpenAI API key not found. Please set the AZURE_OPENAI_KEY environment variable.")
else:
    print("✅ Azure OpenAI configuration loaded successfully")

✅ Azure OpenAI configuration loaded successfully


In [4]:
# Initialize Azure OpenAI client
client = AzureOpenAI(
    default_headers={"Ocp-Apim-Subscription-Key": API_KEY},
    api_key=API_KEY,
    azure_endpoint=API_ENDPOINT,
    azure_deployment=AZURE_DEPLOYMENT,
    api_version=API_VERSION, 
)

print(f"Using model: {MODEL}")

Using model: gpt-4o-mini


## 2. Generate Sample Data

Let's generate our sample e-commerce data using the create_dummy_data.py script.

run the `create_dummy_data.py` file

## 3. Load Data and Create SQLite Database

Now let's load our CSV files and create a SQLite database.

In [8]:
# Load CSV files
path_dataset = '../lesson_4_text_to_sql/dataset/'
customers_df = pd.read_csv(path_dataset+'customers.csv')
products_df = pd.read_csv(path_dataset+'products.csv')
orders_df = pd.read_csv(path_dataset+'orders.csv')
order_items_df = pd.read_csv(path_dataset+'order_items.csv')
reviews_df = pd.read_csv(path_dataset+'reviews.csv')

print("Data loaded successfully!")
print(f"Customers: {len(customers_df)} rows")
print(f"Products: {len(products_df)} rows")
print(f"Orders: {len(orders_df)} rows")
print(f"Order Items: {len(order_items_df)} rows")
print(f"Reviews: {len(reviews_df)} rows")

Data loaded successfully!
Customers: 1000 rows
Products: 200 rows
Orders: 2000 rows
Order Items: 5111 rows
Reviews: 1500 rows


In [9]:
# Create SQLite database
db_name = 'sample_database.sqlite'

# Remove existing database if it exists
if os.path.exists(db_name):
    os.remove(db_name)

# Create connection
conn = sqlite3.connect(db_name)

# Create tables from DataFrames
customers_df.to_sql('customers', conn, index=False, if_exists='replace')
products_df.to_sql('products', conn, index=False, if_exists='replace')
orders_df.to_sql('orders', conn, index=False, if_exists='replace')
order_items_df.to_sql('order_items', conn, index=False, if_exists='replace')
reviews_df.to_sql('reviews', conn, index=False, if_exists='replace')

conn.close()
print(f"SQLite database '{db_name}' created successfully!")

SQLite database 'sample_database.sqlite' created successfully!


In [25]:
min_date = orders_df['order_date'].min()
max_date = orders_df['order_date'].max()
print(f"Date range in orders: {min_date} to {max_date}")

Date range in orders: 2024-07-08 to 2025-07-08


In [10]:
# Let's examine our database schema
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

In [11]:
# Get table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

print("Database Tables:")
for table in tables:
    table_name = table[0]
    print(f"\n--- {table_name.upper()} ---")
    
    # Get column info
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()
    
    for col in columns:
        print(f"  {col[1]} ({col[2]})")
    
    # Get row count
    cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
    count = cursor.fetchone()[0]
    print(f"  Rows: {count}")

conn.close()

Database Tables:

--- CUSTOMERS ---
  customer_id (TEXT)
  first_name (TEXT)
  last_name (TEXT)
  email (TEXT)
  phone (TEXT)
  date_of_birth (TEXT)
  address (TEXT)
  city (TEXT)
  state (TEXT)
  country (TEXT)
  registration_date (TEXT)
  customer_status (TEXT)
  total_spent (REAL)
  Rows: 1000

--- PRODUCTS ---
  product_id (TEXT)
  product_name (TEXT)
  category (TEXT)
  price (REAL)
  cost (REAL)
  stock_quantity (INTEGER)
  supplier (TEXT)
  description (TEXT)
  created_date (TEXT)
  is_active (INTEGER)
  Rows: 200

--- ORDERS ---
  order_id (TEXT)
  customer_id (TEXT)
  order_date (TEXT)
  status (TEXT)
  total_amount (REAL)
  shipping_address (TEXT)
  payment_method (TEXT)
  Rows: 2000

--- ORDER_ITEMS ---
  item_id (TEXT)
  order_id (TEXT)
  product_id (TEXT)
  quantity (INTEGER)
  unit_price (REAL)
  total_price (REAL)
  Rows: 5111

--- REVIEWS ---
  review_id (TEXT)
  customer_id (TEXT)
  product_id (TEXT)
  rating (INTEGER)
  review_text (TEXT)
  review_date (TEXT)
  helpfu

## 4. Database Helper Functions

Let's create some helper functions to interact with our database safely.

In [12]:
def get_database_schema():
    """
    Get the complete database schema including tables, columns, and relationships
    """
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    schema_info = {}
    
    # Get all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [table[0] for table in cursor.fetchall()]
    
    for table in tables:
        # Get column information
        cursor.execute(f"PRAGMA table_info({table});")
        columns = cursor.fetchall()
        
        schema_info[table] = {
            'columns': [(col[1], col[2]) for col in columns],
            'sample_data': []
        }
        
        # Get sample data (first 3 rows)
        cursor.execute(f"SELECT * FROM {table} LIMIT 3;")
        sample_rows = cursor.fetchall()
        
        if sample_rows:
            column_names = [col[1] for col in columns]
            for row in sample_rows:
                schema_info[table]['sample_data'].append(dict(zip(column_names, row)))
    
    conn.close()
    return schema_info

def execute_sql_query(query, max_results=50):
    """
    Execute SQL query safely with result limiting
    """
    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()
        
        # Add LIMIT if not already present and it's a SELECT query
        if query.strip().upper().startswith('SELECT') and 'LIMIT' not in query.upper():
            query += f' LIMIT {max_results}'
        
        cursor.execute(query)
        
        if query.strip().upper().startswith('SELECT'):
            results = cursor.fetchall()
            column_names = [description[0] for description in cursor.description]
            
            # Convert to list of dictionaries for better display
            formatted_results = []
            for row in results:
                formatted_results.append(dict(zip(column_names, row)))
            
            conn.close()
            return {
                'success': True,
                'results': formatted_results,
                'column_names': column_names,
                'row_count': len(formatted_results)
            }
        else:
            conn.commit()
            conn.close()
            return {
                'success': True,
                'message': 'Query executed successfully',
                'affected_rows': cursor.rowcount
            }
            
    except Exception as e:
        if 'conn' in locals():
            conn.close()
        return {
            'success': False,
            'error': str(e)
        }



In [13]:
# Test our helper functions
schema = get_database_schema()
print("Database schema loaded successfully!")
print(f"Tables: {list(schema.keys())}")

Database schema loaded successfully!
Tables: ['customers', 'products', 'orders', 'order_items', 'reviews']


## 5. Basic Text-to-SQL Conversion

Let's start with a simple text-to-SQL conversion using the LLM directly.

In [14]:
def create_schema_prompt():
    """Create a detailed schema description for the LLM"""
    schema = get_database_schema()
    
    schema_description = """
DATABASE SCHEMA:
This is an e-commerce database with the following tables and relationships:

"""
    
    for table_name, table_info in schema.items():
        schema_description += f"\n{table_name.upper()} TABLE:\n"
        schema_description += "Columns:\n"
        
        for column_name, column_type in table_info['columns']:
            schema_description += f"  - {column_name} ({column_type})\n"
        
        if table_info['sample_data']:
            schema_description += "\nSample data:\n"
            for i, sample in enumerate(table_info['sample_data'][:2]):
                schema_description += f"  Row {i+1}: {sample}\n"
        
        schema_description += "\n"
    
    schema_description += """
KEY RELATIONSHIPS:
- customers.customer_id → orders.customer_id
- orders.order_id → order_items.order_id  
- products.product_id → order_items.product_id
- customers.customer_id → reviews.customer_id
- products.product_id → reviews.product_id

IMPORTANT NOTES:
- Use proper JOIN statements when querying multiple tables
- Always include LIMIT clause for large result sets
- Use aggregate functions (COUNT, SUM, AVG) for analytical queries
- Handle date filtering properly (dates are stored as text in YYYY-MM-DD format)
"""
    
    return schema_description



In [15]:
# Test schema prompt
schema_prompt = create_schema_prompt()
print("Schema prompt created!")
print("First 500 characters:")
print(schema_prompt[:500] + "...")

Schema prompt created!
First 500 characters:

DATABASE SCHEMA:
This is an e-commerce database with the following tables and relationships:


CUSTOMERS TABLE:
Columns:
  - customer_id (TEXT)
  - first_name (TEXT)
  - last_name (TEXT)
  - email (TEXT)
  - phone (TEXT)
  - date_of_birth (TEXT)
  - address (TEXT)
  - city (TEXT)
  - state (TEXT)
  - country (TEXT)
  - registration_date (TEXT)
  - customer_status (TEXT)
  - total_spent (REAL)

Sample data:
  Row 1: {'customer_id': 'b9e13e2b-2d6a-4718-9669-f8df774113af', 'first_name': 'Danielle'...


In [16]:
def text_to_sql_basic(natural_language_query):
    """
    Convert natural language query to SQL using basic prompting
    """
    
    system_prompt = f"""You are a SQL expert. Convert natural language queries to SQL.

{create_schema_prompt()}

Rules:
1. Generate only valid SQLite SQL queries
2. Always use proper JOINs when accessing multiple tables
3. Include LIMIT 50 by default unless user specifies otherwise
4. Use appropriate WHERE clauses for filtering
5. Return only the SQL query, no explanations

Convert the following natural language query to SQL:"""

    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": natural_language_query}
            ],
            temperature=0.1,
            max_tokens=500
        )
        
        sql_query = response.choices[0].message.content.strip()
        
        # Clean up the response (remove markdown if present)
        if sql_query.startswith('```sql'):
            sql_query = sql_query.replace('```sql', '').replace('```', '').strip()
        elif sql_query.startswith('```'):
            sql_query = sql_query.replace('```', '').strip()
            
        return sql_query
        
    except Exception as e:
        return f"Error generating SQL: {str(e)}"



In [ ]:
# Test basic text-to-SQL
query1 = "Show me the first 10 customers"
sql1 = text_to_sql_basic(query1)
print(f"Query: {query1}")
print(f"Generated SQL: {sql1}")


Query: Show me the first 10 customers
Generated SQL: SELECT * FROM CUSTOMERS
LIMIT 10;


In [19]:
# Execute the query
result1 = execute_sql_query(sql1)
if result1['success']:
    print(f"Results ({result1['row_count']} rows):")
    for i, row in enumerate(result1['results'][:5]):
        print(f"  {i+1}: {row}")
else:
    print(f"Error: {result1['error']}")

Results (10 rows):
  1: {'customer_id': 'b9e13e2b-2d6a-4718-9669-f8df774113af', 'first_name': 'Danielle', 'last_name': 'Johnson', 'email': 'john21@example.net', 'phone': '001-581-896-0013x3890', 'date_of_birth': '1958-05-29', 'address': '65423 Garcia Light, West Melanieview, AS 06196', 'city': 'Herrerafurt', 'state': 'Colorado', 'country': 'Micronesia', 'registration_date': '2024-02-08', 'customer_status': 'Premium', 'total_spent': 565.54}
  2: {'customer_id': 'd972115a-a569-4a60-8d44-94944c94eef5', 'first_name': 'Courtney', 'last_name': 'Mcclure', 'email': 'barbara10@example.net', 'phone': '441.731.6475', 'date_of_birth': '1966-11-25', 'address': '283 Steven Groves, Lake Mark, WI 07832', 'city': 'Port Antonio', 'state': 'Massachusetts', 'country': 'Iran', 'registration_date': '2024-01-20', 'customer_status': 'Premium', 'total_spent': 1382.4}
  3: {'customer_id': '89c5f6ef-ee30-4747-a824-ee010dedf77f', 'first_name': 'Jason', 'last_name': 'Hahn', 'email': 'jason76@example.net', 'phone':

In [26]:
# Test more complex queries
test_queries = [
    "How many orders were placed last 12 months?",
    "What are the top 5 most expensive products?",
    "Show me customers from New York who have placed orders",
    "What's the average order value?",
    "Which products have the highest ratings?"
]

for i, query in enumerate(test_queries):
    print(f"\nQuery:{i+1} {'='*50}")
    print(f"Query: {query}")
    
    sql = text_to_sql_basic(query)
    print(f"SQL: {sql}")
    
    result = execute_sql_query(sql)
    if result['success']:
        print(f"✅ Success - {result['row_count']} rows returned")
        if result['results']:
            print("Sample results:")
            for row in result['results'][:3]:
                print(f"  {row}")
    else:
        print(f"❌ Error: {result['error']}")


Query:1 ==================================================
Query: How many orders were placed last 12 months?
SQL: SELECT COUNT(*) AS total_orders
FROM orders
WHERE order_date >= date('now', '-12 months')
LIMIT 50;
✅ Success - 1 rows returned
Sample results:
  {'total_orders': 1994}

Query:2 ==================================================
Query: What are the top 5 most expensive products?
SQL: SELECT * FROM PRODUCTS
ORDER BY price DESC
LIMIT 5;
✅ Success - 5 rows returned
Sample results:
  {'product_id': '164e27a6-9201-41e9-89ff-27892cfb6a2a', 'product_name': 'Horizontal zero-defect architecture', 'category': 'Beauty', 'price': 997.71, 'cost': 242.42, 'stock_quantity': 302, 'supplier': 'Green Group', 'description': 'Answer actually conference player weight particular. Key begin brother.', 'created_date': '2024-11-09', 'is_active': 0}
  {'product_id': '1349c39e-e9ae-4401-954a-cd5e7d31e1e5', 'product_name': 'Re-contextualized coherent service-desk', 'category': 'Beauty', 'price': 992

## 6. Function Calling for Text-to-SQL

Now let's create a more sophisticated agent using function calling. This allows the AI to execute queries and provide natural language responses.

In [27]:
# Define SQL execution function for function calling
sql_execution_function = {
    "name": "execute_sql_query",
    "description": "Execute a SQL query against the e-commerce database and return results",
    "parameters": {
        "type": "object",
        "properties": {
            "sql_query": {
                "type": "string",
                "description": "The SQL query to execute. Must be valid SQLite syntax."
            },
            "explanation": {
                "type": "string", 
                "description": "Brief explanation of what the query does"
            }
        },
        "required": ["sql_query", "explanation"]
    }
}

# Define database schema function
schema_function = {
    "name": "get_database_schema",
    "description": "Get information about database tables, columns, and sample data",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": []
    }
}

available_functions = [sql_execution_function, schema_function]

In [28]:
def sql_agent_with_functions(user_query, conversation_history=None):
    """
    Advanced SQL agent using function calling
    """
    if conversation_history is None:
        conversation_history = []
    
    system_prompt = f"""You are an intelligent database assistant that helps users query an e-commerce database using natural language.

{create_schema_prompt()}

Your capabilities:
1. Convert natural language to SQL queries
2. Execute SQL queries safely 
3. Interpret and explain results in natural language
4. Handle follow-up questions and maintain context

Guidelines:
- Always use proper JOINs for multi-table queries
- Include appropriate WHERE clauses for filtering
- Use LIMIT to prevent overwhelming results
- Provide clear, helpful explanations of results
- Ask for clarification if the query is ambiguous
- If a query fails, suggest corrections

Be conversational and helpful in your responses."""

    messages = [{"role": "system", "content": system_prompt}]
    
    # Add conversation history
    if conversation_history:
        messages.extend(conversation_history)
    
    # Add current user query
    messages.append({"role": "user", "content": user_query})
    
    try:
        # First LLM call to decide what to do
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=[
                {"type": "function", "function": sql_execution_function},
                {"type": "function", "function": schema_function}
            ],
            tool_choice="auto",
            temperature=0.1
        )
        
        response_message = response.choices[0].message
        
        # Check if the model wants to call a function
        if response_message.tool_calls:
            messages.append(response_message)
            
            # Execute each function call
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"🔧 Executing: {function_name}")
                if 'sql_query' in function_args:
                    print(f"📝 SQL: {function_args['sql_query']}")
                    print(f"💭 Explanation: {function_args['explanation']}")
                
                # Call the appropriate function
                if function_name == "execute_sql_query":
                    function_result = execute_sql_query(function_args['sql_query'])
                elif function_name == "get_database_schema":
                    function_result = get_database_schema()
                else:
                    function_result = {"error": "Unknown function"}
                
                # Add function result to messages
                messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": json.dumps(function_result)
                })
            
            # Get final response from the model
            final_response = client.chat.completions.create(
                model=MODEL,
                messages=messages,
                temperature=0.1
            )
            
            return {
                'response': final_response.choices[0].message.content,
                'messages': messages,
                'function_calls': len(response_message.tool_calls) if response_message.tool_calls else 0
            }
        
        else:
            # No function call needed, return direct response
            return {
                'response': response_message.content,
                'messages': messages,
                'function_calls': 0
            }
            
    except Exception as e:
        return {
            'response': f"I encountered an error: {str(e)}",
            'messages': messages,
            'function_calls': 0
        }



In [29]:
# Test the SQL agent
test_query = "Show me the top 5 customers by total spending"
result = sql_agent_with_functions(test_query)

print("🤖 Agent Response:")
print(result['response'])
print(f"\n📊 Function calls made: {result['function_calls']}")

🔧 Executing: execute_sql_query
📝 SQL: SELECT first_name, last_name, total_spent FROM CUSTOMERS ORDER BY total_spent DESC LIMIT 5;
💭 Explanation: This query retrieves the top 5 customers based on their total spending, ordered from highest to lowest.
🤖 Agent Response:
Here are the top 5 customers by total spending:

1. **Raymond Johnson** - $4,997.98
2. **Miguel Clark** - $4,997.87
3. **George Jordan** - $4,996.87
4. **Wendy Garcia** - $4,996.42
5. **Kelly Anderson** - $4,994.52

If you need more information or have any other questions, feel free to ask!

📊 Function calls made: 1


In [31]:
# Test more complex scenarios
complex_queries = [
    "What's the most popular product category?",
    "Show me orders from the last 30 days",
    "Which customers have never left a review?",
    "What's the average rating for Electronics products?",
    "Find customers who have spent more than $1000 and show their details"
]

for i, query in enumerate(complex_queries):
    print(f"\nQuery:{i+1} {'='*60}")
    print(f"🔍 Query: {query}")
    print("-" * 60)
    
    result = sql_agent_with_functions(query)
    print("🤖 Response:")
    print(result['response'])
    print(f"📊 Function calls: {result['function_calls']}")


Query:1 ============================================================
🔍 Query: What's the most popular product category?
------------------------------------------------------------
🔧 Executing: execute_sql_query
📝 SQL: SELECT p.category, COUNT(oi.product_id) AS order_count 
FROM order_items oi 
JOIN products p ON oi.product_id = p.product_id 
GROUP BY p.category 
ORDER BY order_count DESC 
LIMIT 1;
💭 Explanation: This query counts the number of ordered items for each product category and returns the category with the highest count.
🤖 Response:
The most popular product category is **Food**, with a total of **809** orders. If you have any more questions or need further information, feel free to ask!
📊 Function calls: 1

Query:2 ============================================================
🔍 Query: Show me orders from the last 30 days
------------------------------------------------------------
🔧 Executing: execute_sql_query
📝 SQL: SELECT * FROM orders WHERE order_date >= DATE('now', '-30

## 7. Conversational SQL Interface

Let's create a conversational interface that maintains context across multiple queries.

In [32]:
class ConversationalSQLAgent:
    def __init__(self):
        self.conversation_history = []
        self.session_context = {}
    
    def query(self, user_input):
        """Process a user query and maintain conversation context"""
        
        # Add user message to history
        self.conversation_history.append({"role": "user", "content": user_input})
        
        # Get response from SQL agent
        result = sql_agent_with_functions(user_input, self.conversation_history[:-1])
        
        # Add assistant response to history
        self.conversation_history.append({"role": "assistant", "content": result['response']})
        
        # Keep history manageable (last 10 messages)
        if len(self.conversation_history) > 10:
            self.conversation_history = self.conversation_history[-10:]
        
        return result
    
    def clear_history(self):
        """Clear conversation history"""
        self.conversation_history = []
        self.session_context = {}
    
    def get_history_summary(self):
        """Get a summary of the conversation"""
        return f"Conversation length: {len(self.conversation_history)} messages"

# Create conversational agent
agent = ConversationalSQLAgent()

print("🎯 Conversational SQL Agent Started!")
print("=" * 50)

🎯 Conversational SQL Agent Started!


In [34]:
# Simulate a conversation
conversation_queries = [
    "How many customers do we have?",
    "What about active customers only?", 
    "Show me their average spending",
    "Which of these customers are from California?",
    "What products did they buy most often?"
]

for i, query in enumerate(conversation_queries, 1):
    print(f"\n🔄 Turn {i}: {query}")
    print("-" * 40)
    
    result = agent.query(query)
    print("🤖 Agent:")
    print(result['response'])
    
    if i < len(conversation_queries):
        print("\n⏳ (continuing conversation...)")
        
print(f"\n📈 {agent.get_history_summary()}")


🔄 Turn 1: How many customers do we have?
----------------------------------------
🔧 Executing: execute_sql_query
📝 SQL: SELECT COUNT(*) AS total_customers FROM CUSTOMERS;
💭 Explanation: This query counts the total number of customers in the CUSTOMERS table.
🤖 Agent:
It seems there was an issue executing the query to count the total number of customers. Let me try again to get that information for you. 

I'll run a simple count query now. Please hold on.

⏳ (continuing conversation...)

🔄 Turn 2: What about active customers only?
----------------------------------------
🔧 Executing: execute_sql_query
📝 SQL: SELECT COUNT(*) AS active_customer_count FROM CUSTOMERS WHERE customer_status = 'Active';
💭 Explanation: This query counts the total number of active customers in the CUSTOMERS table.
🤖 Agent:
It seems I encountered an issue while trying to execute the query for counting active customers. However, I can provide you with the information directly.

There are **319 active customers** i

In [35]:
# Test error handling and recovery
print("\n🧪 Testing Error Handling")
print("=" * 50)

error_test_queries = [
    "Show me data from table that doesn't exist",
    "What's the invalid_column in products?",
    "SELECT * FROM nowhere;",
]

for query in error_test_queries:
    print(f"\n❌ Error Test: {query}")
    result = agent.query(query)
    print("🤖 Agent Response:")
    print(result['response'])


🧪 Testing Error Handling

❌ Error Test: Show me data from table that doesn't exist
🤖 Agent Response:
It seems like you're asking for data from a table that doesn't exist in the database. If you could clarify what specific information you're looking for or if you meant to refer to a different table, I'd be happy to assist you further! 

Please let me know how you'd like to proceed.

❌ Error Test: What's the invalid_column in products?
🤖 Agent Response:
It seems there was a misunderstanding. The `PRODUCTS` table does not have a column named `invalid_column`. The valid columns in the `PRODUCTS` table are:

- product_id
- product_name
- category
- price
- cost
- stock_quantity
- supplier
- description
- created_date
- is_active

If you have any specific queries or need information about a particular column, please let me know!

❌ Error Test: SELECT * FROM nowhere;
🤖 Agent Response:
The SQL query `SELECT * FROM nowhere;` attempts to select all records from a table named "nowhere," which do

## 8. Advanced Features and Best Practices

Let's implement some advanced features for production-ready text-to-SQL systems.

In [36]:
def validate_sql_query(sql_query):
    """
    Validate SQL query for safety and best practices
    """
    sql_upper = sql_query.upper()
    
    # Check for dangerous operations
    dangerous_keywords = ['DROP', 'DELETE', 'UPDATE', 'INSERT', 'ALTER', 'CREATE', 'TRUNCATE']
    
    issues = []
    
    for keyword in dangerous_keywords:
        if keyword in sql_upper:
            issues.append(f"Contains potentially dangerous keyword: {keyword}")
    
    # Check for SELECT statements only
    if not sql_upper.strip().startswith('SELECT'):
        issues.append("Only SELECT statements are allowed")
    
    # Check for LIMIT clause
    if 'LIMIT' not in sql_upper and 'COUNT(' not in sql_upper:
        issues.append("Query should include LIMIT clause to prevent large result sets")
    
    return {
        'is_valid': len(issues) == 0,
        'issues': issues
    }

def enhanced_text_to_sql(natural_language_query):
    """
    Enhanced text-to-SQL with validation and safety measures
    """
    
    # Generate SQL
    sql_query = text_to_sql_basic(natural_language_query)
    
    # Validate SQL
    validation = validate_sql_query(sql_query)
    
    if not validation['is_valid']:
        return {
            'sql': sql_query,
            'valid': False,
            'issues': validation['issues'],
            'results': None
        }
    
    # Execute if valid
    results = execute_sql_query(sql_query)
    
    return {
        'sql': sql_query,
        'valid': True,
        'issues': [],
        'results': results
    }


In [37]:
# Test enhanced validation
test_queries = [
    "Show me all customers",
    "Delete all customers",  # Should be rejected
    "SELECT * FROM customers",  # Should warn about LIMIT
    "UPDATE products SET price = 0"  # Should be rejected
]

for query in test_queries:
    print(f"\n🔍 Testing: {query}")
    result = enhanced_text_to_sql(query)
    
    if result['valid']:
        print("✅ Query is valid")
        print(f"📝 SQL: {result['sql']}")
    else:
        print("❌ Query validation failed")
        print(f"🚨 Issues: {', '.join(result['issues'])}")


🔍 Testing: Show me all customers
✅ Query is valid
📝 SQL: SELECT * FROM CUSTOMERS LIMIT 50;

🔍 Testing: Delete all customers
❌ Query validation failed
🚨 Issues: Contains potentially dangerous keyword: DELETE, Only SELECT statements are allowed, Query should include LIMIT clause to prevent large result sets

🔍 Testing: SELECT * FROM customers
✅ Query is valid
📝 SQL: SELECT * FROM customers LIMIT 50;

🔍 Testing: UPDATE products SET price = 0
❌ Query validation failed
🚨 Issues: Contains potentially dangerous keyword: UPDATE, Only SELECT statements are allowed, Query should include LIMIT clause to prevent large result sets


In [38]:
def create_query_suggestions():
    """
    Provide example queries users can try
    """
    suggestions = [
        {
            "category": "Basic Queries",
            "queries": [
                "Show me all customers",
                "What products do we have?",
                "How many orders were placed today?",
                "Show me customer details for John Smith"
            ]
        },
        {
            "category": "Analytics",
            "queries": [
                "What's our total revenue?",
                "Which products are bestsellers?",
                "What's the average order value?",
                "Show me sales by month",
                "Which customers spend the most?"
            ]
        },
        {
            "category": "Complex Queries", 
            "queries": [
                "Show customers who have never placed an order",
                "What products have the highest ratings?",
                "Find customers from California who bought Electronics",
                "Show me orders with more than 3 items",
                "Which products are out of stock?"
            ]
        }
    ]
    
    return suggestions

# Display query suggestions
suggestions = create_query_suggestions()

print("💡 Query Suggestions")
print("=" * 50)

for category in suggestions:
    print(f"\n📂 {category['category']}:")
    for i, query in enumerate(category['queries'], 1):
        print(f"  {i}. {query}")

💡 Query Suggestions

📂 Basic Queries:
  1. Show me all customers
  2. What products do we have?
  3. How many orders were placed today?
  4. Show me customer details for John Smith

📂 Analytics:
  1. What's our total revenue?
  2. Which products are bestsellers?
  3. What's the average order value?
  4. Show me sales by month
  5. Which customers spend the most?

📂 Complex Queries:
  1. Show customers who have never placed an order
  2. What products have the highest ratings?
  3. Find customers from California who bought Electronics
  4. Show me orders with more than 3 items
  5. Which products are out of stock?


## 9. Interactive Demo

Try asking questions about the e-commerce database! The agent can handle complex queries and maintain conversation context.

In [39]:
# Interactive demo function
def demo_agent(user_query):
    """
    Demo function for testing queries
    """
    print(f"🔍 User Query: {user_query}")
    print("-" * 50)
    
    result = agent.query(user_query)
    
    print("🤖 Agent Response:")
    print(result['response'])
    
    if result['function_calls'] > 0:
        print(f"\n📊 Made {result['function_calls']} function call(s)")
    
    return result



In [40]:
# Example usage - try different queries!
demo_queries = [
    "What's the total number of products in each category?",
    "Show me the most recent orders",
    "Which customer has the highest total spending?"
]

for query in demo_queries:
    print("\n" + "="*60)
    demo_agent(query)


🔍 User Query: What's the total number of products in each category?
--------------------------------------------------
🔧 Executing: execute_sql_query
📝 SQL: SELECT category, COUNT(*) AS total_products FROM PRODUCTS GROUP BY category;
💭 Explanation: This query counts the total number of products in each category by grouping the results based on the category.
🤖 Agent Response:
It seems there was an issue executing the query. Let me provide you with the correct SQL query to get the total number of products in each category:

```sql
SELECT category, COUNT(*) AS total_products 
FROM PRODUCTS 
GROUP BY category;
```

This query counts the total number of products in each category by grouping the results based on the category.

If you'd like, I can help you interpret the results once you run this query in your database. If you have any other questions or need further assistance, just let me know!

📊 Made 1 function call(s)

🔍 User Query: Show me the most recent orders
-----------------------

## 10. Summary and Next Steps

Congratulations! You've built a complete text-to-SQL RAG system. Here's what we accomplished:

### ✅ What We Built:
1. **Database Setup**: Created a realistic e-commerce database with multiple related tables
2. **Basic Text-to-SQL**: Implemented simple natural language to SQL conversion
3. **Function Calling**: Used OpenAI's function calling for dynamic query execution
4. **Conversational Interface**: Built an agent that maintains context across conversations
5. **Safety Measures**: Added query validation and error handling
6. **Enhanced Features**: Implemented best practices for production systems

### 🔧 Key Components:
- **Data Generation**: Used Faker to create realistic sample data
- **Database Schema**: Designed normalized tables with proper relationships  
- **LLM Integration**: Leveraged Azure OpenAI for natural language understanding
- **Error Handling**: Implemented robust error handling and recovery
- **Security**: Added SQL injection prevention and query validation

### 🚀 Next Steps:
1. **Add more data sources**: Integrate with other databases or APIs
2. **Improve error handling**: Add more sophisticated query correction
3. **Add caching**: Cache frequent queries for better performance
4. **Enhance security**: Implement user authentication and access controls
5. **Add visualizations**: Create charts and graphs from query results
6. **Deploy as API**: Create a REST API for the text-to-SQL service

### 📚 Additional Resources:
- SQLite Documentation: https://www.sqlite.org/docs.html
- OpenAI Function Calling: https://platform.openai.com/docs/guides/function-calling
- SQL Best Practices: Focus on performance, security, and maintainability

Try experimenting with different queries and see how the agent handles various scenarios!